In [1]:
import warnings
warnings.filterwarnings('ignore') 

In [2]:
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, LSTM, Input, SimpleRNN, GRU

2024-01-18 17:19:50.008671: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
n_dim = 40

In [3]:
df = pd.read_csv('LOBseries_100ms.csv')
print('Loaded df', df.shape)

Loaded df (2606057, 80)


In [ ]:
df = df.loc[:50000, :n_dim]

In [6]:
tf_dataset = tf.data.Dataset.from_tensor_slices(df)
tf_dataset

2024-01-18 17:20:10.766711: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-18 17:20:10.770759: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


<TensorSliceDataset element_spec=TensorSpec(shape=(40,), dtype=tf.float64, name=None)>

In [7]:
len(tf_dataset)

50000

Create a function where we apply z-score normalization to each input

In [8]:
def z_score(x):
    return (x-tf.math.reduce_mean(x))/tf.math.reduce_std(x)

In [9]:
def get_mid_price(x):
   return (x[0] + x[2])/ 2.0

In [10]:
def generate_labels(current_mid_price, future_mid_prices):
    alpha = 0.002
    mean_future_mid_prices = tf.reduce_mean(future_mid_prices)
    print(mean_future_mid_prices)
    return tf.where(mean_future_mid_prices > current_mid_price + alpha, 2,
                    tf.where(mean_future_mid_prices < current_mid_price - alpha, 0, 1)) 

In [11]:
def generate_labels_no_mid(current_mid_price, future_mid_prices):
    alpha = 0.002
    mean_future_mid_prices = future_mid_prices[-1]
    print(mean_future_mid_prices)
    return tf.where(mean_future_mid_prices > current_mid_price + alpha, 2,
                    tf.where(mean_future_mid_prices < current_mid_price - alpha, 0, 1)) 

Set the horizon to 70 which means we predict the **mean mid price for the next 7 seconds to the future**.

In [22]:
k = 400

In [23]:
mid_prices = tf_dataset.map(get_mid_price, num_parallel_calls= tf.data.AUTOTUNE)
# Create future windows with size the wanted predicted horizon k
future_mid_prices = mid_prices.window(size=k, shift=1, drop_remainder=True)


# Flatmap them and make them numpy iterable iterable 
future_mid_prices = future_mid_prices.flat_map(lambda x: x.batch(k))

# Now we can create the tensorflow dataset type that consists the label for every timestamp
tf_labels = tf.data.Dataset.zip((mid_prices, future_mid_prices)).map(lambda current, future: (generate_labels(current, future)),num_parallel_calls= tf.data.AUTOTUNE)

Tensor("Mean:0", shape=(), dtype=float64)


In [24]:
window_size = 100
batch_size = 32

In [25]:
def make_window_dataset(ds, labels, window_size = 100, shift=1, horizon= 100):
    # Normalize the data by applying z-score to every row
    ds = ds.map(lambda x: z_score(x),num_parallel_calls= tf.data.AUTOTUNE)
    # No split the data to time windows and get rid off the last window size and horizon +2(The +2 is for indexing purposes)
    ds_windows = ds.window(window_size, shift= shift, stride= 1)
    ds_windows = ds_windows.take(ds_windows.cardinality().numpy() - (window_size-1) - (horizon-1) )
    label_windows = labels.window(window_size, shift= shift, stride= 1)
    

    def sub_to_batch(sub):
        return sub.batch(window_size, drop_remainder=True)
    
    # First flatten the dataset to 1 dataset, then map it back with batches of size window_size. Now it is iterable as numpy
    ds_windows = ds_windows.flat_map(sub_to_batch)
    
    label_windows = label_windows.flat_map(sub_to_batch)

    # Now we want to keep only the label from the last lob of every window
    labels = label_windows.map(lambda x: x[-1])

    combined_dataset = tf.data.Dataset.zip((ds_windows, labels))
    return combined_dataset

In [26]:
ds = make_window_dataset(tf_dataset,tf_labels, window_size= window_size, shift = 1, horizon= k)

In [27]:
ds = ds.cache()

In [28]:
length_dataset = 0

for d in ds:
    length_dataset += 1
print(length_dataset)

49502


In [29]:
ds = ds.shuffle(length_dataset)

In [30]:
for data, label in ds:
    print(data.numpy().shape, label.numpy())
    break

(100, 40) 1


In [31]:
neutral = 0
up = 0
down = 0
for data, label in ds:
    if label.numpy() == 1:
        neutral += 1
    elif label.numpy() ==2:
        up += 1
    else:
        down += 1

print(f'{k/10} seconds horizon:')
print('-------------------')
print(f'Price went up {up} times')
print(f'Price went down {down} times')
print(f'Price stayed neutral {neutral} times')

40.0 seconds horizon:
-------------------
Price went up 13790 times
Price went down 19167 times
Price stayed neutral 16545 times


In [32]:
ds = ds.batch(batch_size = batch_size)

In [33]:
for data, label in ds:
    print(data.numpy().shape, label.numpy())
    break

(32, 100, 40) [0 2 1 0 2 0 0 0 1 2 1 2 2 1 2 2 0 2 1 0 2 1 1 0 2 0 0 1 1 1 2 2]


In [34]:
ds = ds.prefetch(tf.data.AUTOTUNE)
ds

<PrefetchDataset element_spec=(TensorSpec(shape=(None, 100, 40), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [35]:
point = int(length_dataset*0.8)
ds_train = ds.take(point)
ds_test = ds.skip(point)

epochs = 10


lr = 0.0001
num_classes = 3

    

- **Class 0** : Price Down
- **Class 1** : Price Neutral
- **Class 2** : Price Up

In [36]:
from LobTransformer import TransformerBlock
from LobFeatures import lob_dilated
from tensorflow.keras.layers import Dropout, Layer
from keras import backend as K

In [37]:
class PositionalEncodingLayer(Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def call(self, x, *args, **kwargs):
        steps, d_model = x.shape.as_list()[-2:]
        ps = np.zeros([window_size, 1], dtype=K.floatx())
        for tx in range(window_size):
            ps[tx, :] = [(2 / (window_size - 1)) * tx - 1]

        ps_expand = K.expand_dims(K.constant(ps), axis=0)
        ps_tiled = K.tile(ps_expand, [K.shape(x)[0], 1, 1])

        x = K.concatenate([x, ps_tiled], axis=-1)
        return x


In [76]:
i = Input(shape=(window_size, n_dim))
x = i
# 5 Dilated Convolution Layers
x = tf.keras.layers.Conv1D(14,kernel_size=2,dilation_rate=1,activation='relu',padding='causal')(x)
x = tf.keras.layers.Conv1D(14,kernel_size=2,dilation_rate=2,activation='relu',padding='causal')(x)
x = tf.keras.layers.Conv1D(14,kernel_size=2,dilation_rate=4,activation='relu',padding='causal')(x)
x = tf.keras.layers.Conv1D(14,kernel_size=2,dilation_rate=8,activation='relu',padding='causal')(x)
x = tf.keras.layers.Conv1D(14,kernel_size=2,dilation_rate=16,activation='relu',padding='causal')(x)

# Layer Norm
x = tf.keras.layers.LayerNormalization()(x)

# Positional Encoding
x = PositionalEncodingLayer()(x)

# Transformer Blocks
tb1 = TransformerBlock('Block1',3, True)
tb2 = TransformerBlock('Block2',3, True)

x= tb1(x)
x= tb2(x)

#MLP
x = tf.keras.layers.Flatten()(x)
x = Dense(64,activation='relu',kernel_regularizer='l2',kernel_initializer='glorot_uniform')(x)

#Dropout
x = Dropout(0.1)(x)

#Output
out = Dense(3, activation='softmax')(x)

model = Model(inputs=i, outputs=out)
model.summary()

Model: "model_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_17 (InputLayer)       [(None, 100, 40)]         0         
                                                                 
 conv1d_72 (Conv1D)          (None, 100, 14)           1134      
                                                                 
 conv1d_73 (Conv1D)          (None, 100, 14)           406       
                                                                 
 conv1d_74 (Conv1D)          (None, 100, 14)           406       
                                                                 
 conv1d_75 (Conv1D)          (None, 100, 14)           406       
                                                                 
 conv1d_76 (Conv1D)          (None, 100, 14)           406       
                                                                 
 layer_normalization_16 (Lay  (None, 100, 14)          28 

In [77]:
model.compile(
    tf.keras.optimizers.Adam(
        learning_rate=0.001,
        beta_1=0.9,
        beta_2=0.999,
        name="Adam",
    ),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

In [78]:
model.fit(ds_train, epochs=epochs, batch_size=batch_size, validation_data = ds_test)

Epoch 1/10
1547/1547 [==============================] - 262s 168ms/step - loss: 1.1959 - sparse_categorical_accuracy: 0.3854
Epoch 2/10
 536/1547 [=========>....................] - ETA: 2:54 - loss: 1.0893 - sparse_categorical_accuracy: 0.3867

KeyboardInterrupt: 